# setuptools

Este un pachet care mărește capabilitățile pachetului `diskutils`. Este folosit pentru a construi și distribui pachete. Este folosit îndeosebi în cazul în care un pachet va avea dependințe care trebuie satisfăcute.

Acest instrument căută, descarcă (HTTP, FTP, Subversion și SourceForge), instalează și actualizează dependințele și scanează automat paginile PyPi pentru a căuta linkurile de descărcare.

Pachetul mai este folosit și pentru a crea Python Eggs, un format de fișier care conține tot ce este necesar pentru a a fi distribuit, ceea ce înseamnă că are posibilitatea de a căuta fișiere și în arhive zip.
Folosirea lui `setuptools` are beneficiul că include automat toate pachetele din arborele proiectului fără a le mai lista individual în `setup.py`.

Setuptools va introduce toate fișierele relevante pentru proiect fără a mai fi necesară constituirea unui fișier `MANIFEST.in`. De asemenea, nu mai este necesară regenerarea forțată a lui `MANIFEST.in` ori de câte ori sursele se modifică.

Extinde utilitarul `diskutils` folosind argumente pasate unui apel `setup()`.

## Instalare

Pentru a instala cea mai nouă versiune a utilitarului `setuptools`, execuți `pip install -U setuptools`.

## Utilizare

```python
from setuptools import setup, find_packages
setup(
    name="HelloWorld",
    version="0.1",
    packages=find_packages(),
)
```

Rularea acestui fragment de cod pus într-un fișier numit `setup.py`, va conduce la crearea unei distribuții din toate pachetele care se află în director.

## Cuvinte cheie pentru setup()

### include_package_data

Dacă valoarea este `True`, va include fișierele de date pe care le va descoperi în directoarele pachetelor care sunt specificate în `MANIFEST.in`.

### exclude_package_data

Este un dicționar care potrivește nume de pachete cu anumite șabloane ce necesită excluderea din directoarele pachetelor.

### package_data

Dacă folosești `include_package_data` și nu ai posibile fișiere generate de setup sau de procesele de build, nu este nevoie.

### zip_safe

Este un flag `True` sau `False` care indică dacă un proiect poate fi instalat în siguranță dintr-o arhivă zip.

### install_requires

Este un șir de caractere sau o listă de șiruri care specifică ce trebuie să instaleze alte distribuții în momentul în care o instalează pe aceasta.

TODO: De completat!

## Folosirea lui find_packages()

Această meodă `setuptools.find_packages()` este folosită pentru a gestiona căutarea pachetelor care sunt necesare în cazul unor proiecte de mari dimensiuni. Returnează o listă de pachete gata de a fi folosite ca arumentul `packages` pentru `setup()`.

Metoda ia numele unui director în care se află sursele și două liste cu denumirile pachetelor cu scopul de a le include sau a le exclude. Dacă nu este menționat niciun director, acesta va fi chiar cel în care rulează scriptul de `setup`. Cel mai adesea, veți vedea că proiectele folosesc directoarele `src` sau `lip` drept rădăcini pentru arborele surselor: `package_dir={'':'src'}` în argumentele lui `setup()`.

Metoda va traversa directorul țintă pentru a căuta pachetele Python (cele care au fișierul `__init__`).

Menționând anumite șabloane, poți elimina ceea ce nu dorești să se afle în distribuție. De exemplu, `find_packages(exclude=["*.trial"]` va elimina pachetele a căror nume se încheie cu `trial`.

## find_namespace_packages()

Începând cu versiunea 3.3+ a lui Python, este disponibilă și această metodă pentru a slecta toate pachetele care se află într-un namespace. Metoda poate fi considerată o variată la `find_packages`, dar care o extinde pentru a beneficia și de prevederile [PEP 420 -- Implicit Namespace Packages](https://www.python.org/dev/peps/pep-0420/).

Pentru o structură a codului care respectă PEP420, putem avea următorul exemplu oferit de documentație.

```text
├── namespace
│   └── mypackage
│       ├── __init__.py
│       └── mod1.py
├── setup.py
└── tests
    └── test_mod1.py
```

Scriptul de instalare poate să aibă următoarea formă.


```python
from setuptools import setup, find_namespace_packages
setup(
    name="HelloWorld",
    version="0.1",
    packages=find_namespace_packages(),
)
```

O altă formă de aranjare a structurii proiectului poate fi una care să aibă drept rădăcină un director `src`.

```text
├── setup.py
├── src
│   └── namespace
│       └── mypackage
│           ├── __init__.py
│           └── mod1.py
└── tests
    └── test_mod1.py
```

În acest caz, scriptul de `setup()` poate avea următoarea formă.

```python
setup(name="namespace.mypackage",
      version="0.1",
      package_dir={'': 'src'},
      packages=find_namespace_packages(where='src'))
```

## Declararea dependințelor

În momentul în care este instalat un pachet, `setuptools` permite instalarea tuturor dependințelor. Informațiile privind dependințele sunt incluse în Python Eggs.

Pentru a specifica dependințele, este nevoie de o anumită sintaxă. Această sintaxă constă din numele proiectului așa cum se găsește acesta în PyPi, urmat fiind de o listă separată prin virgule de `extras` menționate între paranteze pătrate, opțional urmate de o listă de specificatori ai versiunii separați prin virgule.

```python
PEAK[FastCGI, reST]>=0.5a4
```

Un specificator de versiune folosește unul din operatorii `<`, `>`, `<=`, `>=`, `==` sau `!=` urmați de specificatorii versiunii.

Dependințele sunt specificate în argumentul `install_requires`. Acesta acceptă o listă.

La momentul când proiectul este instalat prin `setup.py install` sau `setup.py develop`, toate dependințele care încă nu sunt instalate vor fi căutate via PyPI, vor fi descărcate, dacă este necesar, vor trece printr-un build și vor fi instalate.

Toate scripturile din program vor fi instalate cu o împachetare care verifică disponibilitatea anumitor pachete la momentul rulării. Aceeași împachetare se asigură de faptul că sunt adăugate la `sys.path` versiunile corecte. Distribuțiile care folosesc Python Egg vor include un fișier de metadate care vor lista dependințele.

Dacă declari dependințele în `setup.py`, nu va mai trebui să folosești funcția `require()` în scripturi sau modulele pe care le-ai dezvoltat. Pentru a beneficia de acest lucru trebuie să instalezi proiectul folosind `setup.py`.

## Dependințe care nu se află în PyPi

Dacă proiectul depinde de pachete care nu există în PyPI, poți să le descarci ca egg (formatul distutils sdist), ca fișiere `py` unice sau ca resurse existente în sisteme de versionare.

Linkurile către resurse pot fi menționate în argumentul `dependency_links` al lui `setup`. Aceste URL-uri trebuie să conducă direct la resursă, la o pagină web care conține linkuri directe către resurse sau către un depozit.

Dacă depinzi de un fișier `.py`, va trebui să adaugi la URL un sufix: `"#egg=project-version"` (fă scape la liniuțele din nume sau din versiune înlocuidu-le cu underscore).

```python
vcs+proto://host/path@revision#egg=project-version
```

Unde `vcs+proto` poate fi `svn+URL` (Subversion), `git+URL` (Git) și `hg+URL` pentru Mercurial.

```python
setup(
    ...
    dependency_links=[
        "http://peak.telecommunity.com/snapshots/"
    ],
)
```


## Declararea unor pachete opționale - extras

Un proiect poate avea dependințe care sunt pe lista celor recomandate a fi instalate, dacă dorești să beneficiezi de toate funcționalitățile unui proiect. Aceste pachete suplimentare opționale sunt numite `extras`. Argumentul este `extras_require` și acceptă un dicționar prin care se face o asociere de chei cu pachete suplimentare necesare instalării.

Documentația oferă un exemplu în care un proiect are nevoie să ofere suport pentru PDF-uri și pentru reST (reStructuredText).

```python
setup(
    name="Project-A",
    ...
    extras_require={
        'PDF':  ["ReportLab>=1.2", "RXP"],
        'reST': ["docutils>=0.3"],
    }
)
```

Trebuie menționat faptul că aceste pachete nu vor fi instalate dacă nu sunt cerute ca dependințe.
Aceste pachete extra pot fi folosite de unele puncte de intrare (**entry points**), fiind astfel considerate drept dependințe dinamice.

```python
setup(
    name="Project-A",
    ...
    entry_points={
        'console_scripts': [
            'rst2pdf = project_a.tools.pdfgen [PDF]',
            'rst2html = project_a.tools.htmlgen',
            # more script entry points ...
        ],
    }
)
```

În cazul prezentat de manual, în cazul în care în proiectul `Project-A` avem un script `rst2pdf`, se va rezolva dependința în cazul în care acest script are nevoie să creeze PDF-uri.

Mai putem avea cazul ceva mai complex în care un posibil `Project-B` poate avea nevoie de `Project-A`, care să aibă suport pentru PDF-uri. O astfel de dependință ar putea fi rezolvată menționând la `install_requires` numele dependinței și între paranteze pătrate, ce trebuie acea dependință să folosească la rândul ei.

```python
setup(
    name="Project-B",
    install_requires=["Project-A[PDF]"],
    ...
)
```

În cazul de mai sus, va fi instalat `Project-A` ca dependință, dar se va cere lui `Project-A` să instaleze odată cu sine și ceea ce acesta avea menționat la `extras_require` pe cheia `PDF`, adică cele două pachete care implementează suportul pentru PDF-uri: `"ReportLab>=1.2", "RXP"`. Chiar datcă `Project-A` era deja instalat, se va mai face instalarea acestuia, dar de data aceasta și a dependințelor din `extras_require`.

Poate fi considerat un mod destul de elegant de a incapsula dependințe de care poate un oachet să aibă nevoie sau nu, dar cu certitudine poate va avea nevoie atunci când este instalat cu altele care-l cer, dar cu tot ce poate oferi (*downstream dependencies*). Partea elegantă vine din faptul că pot fi agregate mai multe dependințe sub un singur nume (*feature name*). În cazul nostru, `PDF`. Astfel, alte pachete care vor avea nevoie de suport PDF, pur și simplu nu vor trebui să știe câte pachete suplimentare vor fi instalate, ci doar numele care generic sub care sunt menționate. Acest lucru permite și remodelarea dependințelor de sub `PDF`, dacă ceva mai bun apare, sau alte versiuni, etc.

În cazul în care `Project-A` nu mai are nevoie de pachetele suplimentare pentru PDF, dar `Project-B`, care cere pe A, solicită în scriptul de instalare acest suport, pentru a nu rupe dependințele, se poate trece la `extras_require` o cheie care pur și simplu, va fi goală.

```python
setup(
    name="Project-A",
    ...
    extras_require={
        'PDF':  [],
        'reST': ["docutils>=0.3"],
    }
)
```

## Declararea unor dependințe specifice unei platforme

Uneori o anumită dependință trebuie instalată în funcție de sistemul de operare pe care rulează aplicația.

```python
setup(
    name="Project",
    ...
    install_requires=[
        'enum34;python_version<"3.4"',
        'pywin32 >= 1.0;platform_system=="Windows"'
    ]
)
```

## Includerea unor pachete de date

Aceste fișiere de date se instalează, de regulă în rădăcina pachetului pentru uzul său. Pentru a instala aceste fișiere, va trebui să pasezi lui `setup()`, argumentul `include_package_data` cu valoarea `True`.

```python
from setuptools import setup, find_packages
setup(
    ...
    include_package_data=True
)
```

Fișierele de date sunt specificate în `MANIFEST.in`. Dacă se dorește un control mult mai fin asupra resurselor care vor fi incluse în pachetele instalate, se poate ajunge la un nivel mai mare de granularitate folosind argumentul `package_data` pasat lui `setup()`. Acest nu este nimic mai mult decât un obiect.

```python
from setuptools import setup, find_packages
setup(
    ...
    package_data={
        # Dacă vreun pachet include fișiere *.txt sau *.rst files, include-le
        '': ['*.txt', '*.rst'],
        # Dacă pachetul 'hello' conține orice fișier *.msg include-l
        'hello': ['*.msg'],
    }
)
```

Documentația oferă exemplul unui pachet aflat în directorul `src`.

```text
setup.py
src/
    faceva/
        __init__.py
        faceva.txt
        data/
            nistedate.dat
            plusaltele.dat
```

Un fișier de `setup()` ar putea avea argumentele astfel.

```python
from setuptools import setup, find_packages
setup(
    ...
    packages=find_packages('src'),  # include toate pachetele din src
    package_dir={'':'src'},         # îi spune lui distutilscă pachetele sunt în src

    package_data={
        # Davă vreun pachet conține vreun fișier *.txt include-l
        '': ['*.txt'],
        # Include orice fișier *.dat din directorul 'data' al pachetului 'faceva'
        'faceva': ['data/*.dat'],
        # exclude README.txt din toate pachetele
        exclude_package_data={'': ['README.txt']},
    }
)
```

Reține faptul că menționarea unei chei goale (`''`) în obiectul de mapping înseamnă că se aplică tuturor pachetelor. În cazul în care nu mai folosești un fișier de date, dar l-ai inclus în proiect, acesta va fi considerat a fi orfan și va trebuie să rulezi `setup.py clean --all` pentru a-l elimina.



## Descoperirea dinamică a serviciilor și pluginurilor

Utilitarul `setuptools` permite crearea de biblioteci de cod (*libraries*) care se pot lega dinamic (*plug in*) la alte aplicații extensibile și framework-uri prin realizarea unor *puncte de intrarea* - `entry points`  ale propriului proiect care pot fi importate de alte aplicații sau framework-uri.

De exemplu, dacă un instrument de blogging dorește să permită conectarea unor pluginuri care să permită traducerea pentru diferite tipuri de fișiere produse de blog. Framework-ul ar putea defini un „entry point group” numit blogtool.parsers. Aceasta ar trebui să permită pluggin-uri pentru a defini entry points pentru extensiile de fișiere pe care le suportă.

Dacă vei crea un proiect care se conectează la o aplicație care există sau la un framework, va trebui să cunoști care sunt grupurile entry point-urilor definite de acea aplicație sau framework.

Entry point-urile în scripturile de setup. Entry point-urile sunt înregistrate în fișiere .rst.

```python
setup(
    # ...
    entry_points={'blogtool.parsers': '.rst = some_module:SomeClass'}
)

setup(
    # ...
    entry_points={'blogtool.parsers': ['.rst = some_module:a_func']}
)

setup(
    # ...
    entry_points="""
        [blogtool.parsers]
        .rst = some.nested.module:SomeClass.some_classmethod [reST]
    """,
    extras_require=dict(reST="Docutils>=0.3.5")
)
```

Argumentul `entry_points` al funcției setup() acceptă fie un string cu secțiuni tipic fișierelor .ini sau un dicționar cu nume ale entry point group-uri menționate ca string-uri sau liste de string-uri care conțin specificatori ai entry point-uri.

Un specificator al unui entry point constă dinstr-un nume și o valoare separate de semnul `=`.

An entry point specifier consists of a name and value, separated by an = sign. The value consists of a dotted module name, optionally followed by a : and a dotted identifier naming an object within the module. It can also include a bracketed list of “extras” that are required for the entry point to be used. When the invoking application or framework requests loading of an entry point, any requirements implied by the associated extras will be passed to pkg_resources.require(), so that an appropriate error message can be displayed if the needed package(s) are missing. (Of course, the invoking app or framework can ignore such errors if it wants to make an entry point optional if a requirement isn’t installed.)

## Distribuirea unui proiect bazat pe setuptools

Înainte de a porni, trebuie să te asiguri că ai actualizat pachetele de lucru.

```bash
python3 -m pip install --user --upgrade setuptools wheel
```

Pentru a contrui proiectul, rulezi comanda.

```bash
python3 setup.py sdist bdist_wheel
```

Drept efect, în directorul `dist` vor fi create arhivele distribuției. Pentru a încărca arhivele pe PyPI, va trebui să ai instalat `twine`.

```bash
twine upload --repository-url https://test.pypi.org/legacy/ dist/*
```

Pentru a instala noul pachet creat, poți folosi `pip`.

```bash
python3 -m pip install --index-url https://test.pypi.org/simple/ pkg_nou
```

## Configurarea lui setup() folosind setup.cfg

Fișierele `setup.cfg` definesc metadatele pachetului și alte opțiuni care sunt introduse în `setup()`.

```text
[metadata]
name = my_package
version = attr: src.VERSION
description = My package description
long_description = file: README.rst, CHANGELOG.rst, LICENSE.rst
keywords = one, two
license = BSD 3-Clause License
classifiers =
    Framework :: Django
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.5

[options]
zip_safe = False
include_package_data = True
packages = find:
scripts =
  bin/first.py
  bin/second.py
install_requires =
  requests
  importlib; python_version == "2.6"

[options.package_data]
* = *.txt, *.rst
hello = *.msg

[options.extras_require]
pdf = ReportLab>=1.2; RXP
rest = docutils>=0.3; pack ==1.1, ==1.3

[options.packages.find]
exclude =
    src.subpackage1
    src.subpackage2

[options.data_files]
/etc/my_package =
    site.d/00_default.conf
    host.d/00_default.conf
data = data/img/logo.png, data/svg/icon.svg
```
